In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import cv2
import numpy as np
import os
from tqdm import tqdm
input_folder =  "/content/drive/MyDrive/dp/data/sabbir"
output_folder =  "/content/drive/MyDrive/dp/data/CLAHE_ConstantAdjustment_Gamma_HistogramEqualization"

os.makedirs(output_folder, exist_ok=True)

def preprocess_pipeline(img_path, save_folder):

    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return

    filename = os.path.basename(img_path).split('.')[0]

    # ---- Step 1: CLAHE ----
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    img = clahe.apply(img)

    # ---- Step 2: Constant Adjustment ----
    alpha, beta = 1.2, 20
    img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

    # ---- Step 3: Gamma Correction ----
    gamma = 1.5
    gamma_table = np.array([((i / 255.0) ** (1.0/gamma)) * 255
                            for i in np.arange(256)]).astype("uint8")
    img = cv2.LUT(img, gamma_table)

    # ---- Step 4: Histogram Equalization ----
    img = cv2.equalizeHist(img)
    cv2.imwrite(os.path.join(save_folder, f"{filename}_processed.png"), img)



all_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png','.jpg','.jpeg'))]

for file in tqdm(all_files):
    preprocess_pipeline(os.path.join(input_folder, file), output_folder)

print("✅ All images processed and saved in:", output_folder)
